In [ ]:
import pathlib as pl
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import matplotlib.ticker as mticker

import pandas as pd

from shapely.geometry import LineString, Polygon

from scipy.spatial.distance import cdist

import xarray as xa

import flopy
from flopy.discretization import VertexGrid
from flopy.utils.triangle import Triangle
from flopy.utils.voronoi import VoronoiGrid
import flopy.plot.styles as styles

In [ ]:
from defaults import (
    font_dict,
    arrowprops,
    grid_dict,
    lake_cmap,
    contour_label_dict,
    river_dict,
    clabel_dict,
    sv_contour_dict,
    intersection_rgba,
    intersection_cmap,
    clay_cmap,
    circle_function,
    string2geom,
    densify_geometry,
)

In [ ]:
save_figures = True
sample_frequency = "monthly"  # monthly or annual

In [ ]:
if save_figures:
    fig_path = pl.Path("../../models/figures/")
    fig_path.mkdir(parents=True, exist_ok=True)

In [ ]:
base_ws = "../../models/synthetic-valley-vg/"
name = "sv"

### Conversion factors

In [ ]:
in2ft = 1.0 / 12.0
ft2m = 1.0 / 3.28081
ft3tom3 = 1.0 * ft2m * ft2m * ft2m
ftpd2cmpy = 1000.0 * 365.25 * ft2m
mpd2cmpy = 100.0 * 365.25
mpd2inpy = 12.0 * 365.25 * 3.28081

### Temporal data

In [ ]:
path = pl.Path(f"../data/temporal_data_{sample_frequency}.parquet")
temporal_df = pd.read_parquet(path)

### Define the stress period data

In [ ]:
idx_end_calibration = 0
if sample_frequency == "monthly":
    idx_end_period2 = 120
    idx_end_period3 = 240
elif sample_frequency == "annual":
    idx_end_period2 = 10
    idx_end_period3 = 20
else:
    raise ValueError(f"invalid sample_frequency: '{sample_frequency}'")

start_date = pd.to_datetime("1962-01-01 00:00:00")
start_date_time = str(start_date).replace(" ", "T")

end_calibration = temporal_df.index[idx_end_calibration]
end_period_two = temporal_df.index[idx_end_period2]
end_period_three = temporal_df.index[idx_end_period2]

end_periods = [end_calibration, end_period_two, end_period_three]

totim_end = [float((end_calibration - start_date).days)]
totim_end += [float((end_period_two - start_date).days)]
totim_end += [float((end_period_three - start_date).days)]

end_periods, totim_end

In [ ]:
perlen = temporal_df["perlen"].values[: idx_end_period2 + 1]
nstp = [1 for idx in range(len(perlen))]
tsmult = [1.0 for idx in range(len(perlen))]

nper = len(perlen)
tdis_ds = [(p, n, t) for p, n, t in zip(perlen, nstp, tsmult)]

### Load raster data

In [ ]:
nc_path = pl.Path("../data/synthetic_valley_truth.nc")
nc_ds = xa.open_dataset(nc_path)
nc_ds

In [ ]:
kaq = flopy.utils.Raster.load("./data/kaq_3d_SI.tif")
kclay = flopy.utils.Raster.load("./data/k_clay_SI.tif")
top_base = flopy.utils.Raster.load("./data/top_SI.tif")
bot = flopy.utils.Raster.load("./data/bottom_3d_SI.tif")
lake_location = flopy.utils.Raster.load("./data/lake_location_SI.tif")
clay_location = flopy.utils.Raster.load("./data/clay_location_SI.tif")
runoff_map = flopy.utils.Raster.load("./data/runoff_routing_SI.tif")

In [ ]:
crs = "EPSG:8377"

### define the features (in feet)

In [ ]:
sv_boundary = """0.0 0.0
0.0 20000.0
12500.0 20000.0
12500.0 0.0"""

In [ ]:
river = """4250.0 8750.0 
4250.0 0.0"""

In [ ]:
river_box = """3500.0 0.0
3500.0 9500.0
5000.0 9500.0
5000.0 0.0"""

In [ ]:
wells = """7250. 17250.
7750. 2750.
2750 3750."""

In [ ]:
lake = """1500. 18500.
3500. 18500.
3500. 15500.
4000. 15500.
4000. 14500.
4500. 14500.
4500. 12000.
2500. 12000.
2500. 12500.
2000. 12500.
2000. 14000.
1500. 14000.
1500. 15000.
1000. 15000.
1000. 18000.
1500. 18000."""

In [ ]:
# in meters
boundary_refinement = 100.0
river_refinement = 25.0
lake_refinement = 30.0
max_boundary_area = boundary_refinement * boundary_refinement
max_river_area = river_refinement * river_refinement
max_lake_area = lake_refinement * lake_refinement

In [ ]:
boundary_polygon = string2geom(sv_boundary, conversion=ft2m)
print("len boundary", len(boundary_polygon))
bp = np.array(boundary_polygon)
bp_densify = np.array(densify_geometry(bp, boundary_refinement))

In [ ]:
river_polyline = string2geom(river, conversion=ft2m)
sg = np.array(river_polyline)
sg_densify = np.array(densify_geometry(sg, river_refinement))

In [ ]:
river_boundary = string2geom(river_box, conversion=ft2m)
rb = np.array(river_boundary)
rb_densify = np.array(densify_geometry(rb, river_refinement))

In [ ]:
lake_polygon = string2geom(lake, conversion=ft2m)
lake_plot = string2geom(lake, conversion=ft2m)
lake_plot += [lake_plot[0]]
lake_plot = np.array(lake_plot)
lp = np.array(lake_polygon)
lp_densify = np.array(densify_geometry(lp, lake_refinement))

In [ ]:
well_points = string2geom(wells, conversion=ft2m)
wp = np.array(well_points)

#### simple plotting functions

In [ ]:
def plot_wells(ax=None, ms=None):
    if ax is None:
        ax = plt.gca()
    ax.plot(wp[:, 0], wp[:, 1], "ro", ms=ms)
    return ax

In [ ]:
def plot_river(
    ax=None,
):
    if ax is None:
        ax = plt.gca()
    ax.plot(sg_densify[:, 0], sg_densify[:, 1], **river_dict)
    return ax

In [ ]:
def plot_lake(ax=None, lw=0.5, color="cyan", marker=None, densify=False):
    if ax is None:
        ax = plt.gca()
    if densify:
        arr = lp_densify
    else:
        arr = lake_plot
    ax.plot(arr[:, 0], arr[:, 1], ls="-", color=color, lw=lw, marker=marker)
    return ax

In [ ]:
def plot_circles(ax=None, radius=1.0, lw=0.5, color="red"):
    if ax is None:
        ax = plt.gca()
    for pt in wp:
        center = (pt[0], pt[1])
        pts = circle_function(center=center, radius=radius)
        ax.plot(pts[:, 0], pts[:, 1], ls=":", color=color, lw=lw)
    return ax

### use Triangle to create mesh

In [ ]:
nlay = 5

In [ ]:
temp_path = pl.Path(".temp/triangle_data")
temp_path.mkdir(parents=True, exist_ok=True)

In [ ]:
maximum_area = 150.0 * 150.0
well_dv = 300.0

In [ ]:
tri = Triangle(
    angle=30,
    nodes=sg_densify,
    model_ws=temp_path,
)
tri.add_polygon(bp_densify)
tri.add_polygon(rb_densify)
tri.add_polygon(lp_densify)
tri.add_region((10, 10), attribute=10, maximum_area=max_boundary_area)
tri.add_region(
    (3050.0, 3050.0),
    attribute=10,
    maximum_area=max_boundary_area,
)
tri.add_region((900.0, 4600.0), attribute=11, maximum_area=max_lake_area)
tri.add_region((1200.0, 150.0), attribute=10, maximum_area=max_river_area)
for idx, w in enumerate(wp):
    center = (w[0], w[1])
    tri.add_polygon(circle_function(center=center, radius=100.0))
    tri.add_region(center, attribute=idx, maximum_area=500.0)
tri.build(verbose=False)

### convert triangular mesh to a voronoi grid

In [ ]:
vor = VoronoiGrid(tri)

### create a flopy Vertex grid to use for the model

In [ ]:
gridprops = vor.get_gridprops_vertexgrid()
idomain = np.ones((1, vor.ncpl), dtype=int)
voronoi_grid = VertexGrid(**gridprops, nlay=1, idomain=idomain)

In [ ]:
shape2d = (nlay, vor.ncpl)

### intersect the rasters with the vertex grid

In [ ]:
top_vg = top_base.resample_to_grid(
    voronoi_grid,
    band=top_base.bands[0],
    method="linear",
    extrapolate_edges=True,
)

In [ ]:
bot_vg = [
    bot.resample_to_grid(
        voronoi_grid,
        band=bot.bands[idx],
        method="linear",
        extrapolate_edges=True,
    )
    for idx in range(nlay)
]

In [ ]:
lake_cells_vg = lake_location.resample_to_grid(
    voronoi_grid,
    band=lake_location.bands[0],
    method="nearest",
    extrapolate_edges=True,
)

In [ ]:
kaq_vg = [
    kaq.resample_to_grid(
        voronoi_grid,
        band=kaq.bands[idx],
        method="nearest",
        extrapolate_edges=True,
    )
    for idx in range(nlay)
]

In [ ]:
kclay_vg = kclay.resample_to_grid(
    voronoi_grid,
    band=kclay.bands[0],
    method="nearest",
)

In [ ]:
clay_location_vg = clay_location.resample_to_grid(
    voronoi_grid,
    band=clay_location.bands[0],
    method="nearest",
    extrapolate_edges=True,
)

In [ ]:
runoff_map_vg = runoff_map.resample_to_grid(
    voronoi_grid,
    band=runoff_map.bands[0],
    method="nearest",
    extrapolate_edges=True,
)

In [ ]:
mv = flopy.plot.PlotMapView(modelgrid=voronoi_grid)
v = mv.plot_array(runoff_map_vg, cmap="viridis")
plt.colorbar(v, label="Runoff Routing Map Values")
mv.ax.set_title("Runoff Routing Map Resampled to Voronoi Grid")

### Set idomain for model

Pass through cells where the clay is missing

In [ ]:
idomain_2 = np.ones(kclay_vg.shape, dtype=int)
idomain_2[clay_location_vg == 1] = -1
idomain_vg = [1] * nlay
idomain_vg[2] = idomain_2

### Modify bottom of model

Set bottom of layer 2 to the bottom of layer 3 where the clay is missing

In [ ]:
bot_vg[1][clay_location_vg == 1] = bot_vg[2][clay_location_vg == 1]

### Modify hydraulic properties for model

Set Kh and Kv to clay values where clay is present. Also set Ss and Sy to larger values where clay is present.

In [ ]:
k11_vg = np.array(kaq_vg)
k33_vg = 0.25 * k11_vg
k11_vg[2][clay_location_vg == 1] = kclay_vg[clay_location_vg == 1]
k33_vg[2][clay_location_vg == 1] = kclay_vg[clay_location_vg == 1]

In [ ]:
porosity_2 = np.full(kclay_vg.shape, 0.2, dtype=float)
porosity_2[clay_location_vg == 1] = 0.4
porosity_vg = [0.2] * nlay
porosity_vg[2] = porosity_2

In [ ]:
porosity_2.min(), porosity_2.max()

In [ ]:
ss_2 = np.full((vor.ncpl,), 5e-5, dtype=float)
ss_2[clay_location_vg == 1] = 5e-3
ss_vg = [1e-5] * nlay
ss_vg[2] = ss_2

In [ ]:
icelltype_vg = [1] + [0] * (nlay - 1)

### Create list for the bottom of each layer

In [ ]:
botm = [bot_vg[k].copy() for k in range(nlay)]

### create a model grid for the lake

In [ ]:
lake_grid_top = np.full((vor.ncpl), 50.0, dtype=float)
lake_vg_grid = flopy.discretization.VertexGrid(
    **gridprops,
    nlay=1,
    idomain=idomain,
    top=lake_grid_top,
    botm=top_vg.reshape(1, vor.ncpl),
)

### plot the resampled rasters

In [ ]:
two_panel_figsize = (17.15 / 2.541, 0.8333 * 17.15 / 2.541)
one_panel_figsize = (8.25 / 2.541, 13.25 / 2.541)
six_panel_figsize = (17.15 / 2.541, 1.4 * 0.8333 * 17.15 / 2.541)

### intersect features with the grid

In [ ]:
ixs = flopy.utils.GridIntersect(voronoi_grid, method="vertex")
sg_result = ixs.intersect(LineString(sg_densify), sort_by_cellid=False)

In [ ]:
sg_result.shape, sg_result["lengths"].sum(), sg_result.dtype.names

In [ ]:
cellids = sg_result["cellids"]
xc, yc = (
    voronoi_grid.get_xcellcenters_for_layer(0),
    voronoi_grid.get_ycellcenters_for_layer(0),
)
sfr_points = np.array([(xc[cellid], yc[cellid]) for cellid in cellids])  # for mover

### forcing data

In [ ]:
rainfall = 0.00821918 * ft2m
evaporation = 0.0062296 * ft2m
net_rainfall = 0.003641 * ft2m

In [ ]:
rainfall, evaporation

In [ ]:
rainfall * mpd2cmpy, evaporation * mpd2cmpy, net_rainfall * mpd2cmpy

In [ ]:
rainfall * mpd2inpy, evaporation * mpd2inpy, net_rainfall * mpd2inpy

In [ ]:
temporal_df.columns

In [ ]:
rch_ts = "PRCP (Inches)"
recharge_spd = {}
for n in range(nper):
    row = temporal_df.iloc[n]
    recharge = float(row[rch_ts]) * in2ft * ft2m
    recharge_spd[n] = recharge

In [ ]:
evt_spd = {}
evt_ts = "land et (inches)"
for n in range(nper):
    row = temporal_df.iloc[n]
    pet = float(row[evt_ts]) * in2ft * ft2m
    evt_spd[n] = pet

## Build SFR, LAK, DRN, and WEL datasets

SFR datasets

In [ ]:
sfr_unit_conversion = 1.0 * 86400.0  # 1.486 * 86400.0
sfr_time_conversion = 86400.0

In [ ]:
sfr_plt_array = np.zeros(voronoi_grid.ncpl, dtype=int)

In [ ]:
sfr_nodes = np.arange(0, sg_result.shape[0])
gwf_nodes = sg_result["cellids"][::-1]
sfr_lengths = sg_result["lengths"][::-1]

In [ ]:
total_cond = 1800000.0 * ft3tom3
sfr_width = 10.0 * ft2m
sfr_bedthick = 1.0 * ft2m
sfr_hk = total_cond * sfr_bedthick / (sfr_width * sfr_lengths.sum())
total_cond, sfr_hk

In [ ]:
b0, b1 = -0.3 * ft2m, -2.05 * ft2m
sfr_slope = -0.0002
cum_dist = np.zeros(sfr_nodes.shape, dtype=float)
cum_dist[0] = 0.5 * sfr_lengths[0]
for idx in range(1, sfr_nodes.shape[0]):
    cum_dist[idx] = cum_dist[idx - 1] + 0.5 * (sfr_lengths[idx - 1] + sfr_lengths[idx])
sfr_bot = b0 + sfr_slope * cum_dist
cum_dist[-1], sfr_lengths.sum(), sfr_lengths[-1], sfr_bot[-1]

In [ ]:
sfr_conn = []
for idx, node in enumerate(sfr_nodes):
    iconn = [node]
    if idx > 0:
        iconn.append(sfr_nodes[idx - 1])
    if idx < sfr_nodes.shape[0] - 1:
        iconn.append(-sfr_nodes[idx + 1])
    sfr_conn.append(iconn)

In [ ]:
# <rno> <cellid(ncelldim)> <rlen> <rwid> <rgrd> <rtp> <rbth> <rhk> <man> <ncon> <ustrf> <ndv>
sfr_iflowface = -1.0
sfrpak_data = []
for idx, (cellid, rlen, rtp) in enumerate(zip(gwf_nodes, sfr_lengths, sfr_bot)):
    sfr_plt_array[cellid] = 1
    sfrpak_data.append(
        (
            idx,
            (
                0,
                cellid,
            ),
            rlen,
            sfr_width,
            -sfr_slope,
            rtp,
            sfr_bedthick,
            sfr_hk,
            0.030,
            (len(sfr_conn[idx]) - 1),
            1.0,
            0,
            sfr_iflowface,
        )
    )
sfrpak_data[-2:]

In [ ]:
sfr_spd = {}
rain_tag = "PRCP (Inches)"
pet_tag = "lake et (inches)"
for n in range(nper):
    row = temporal_df.iloc[n]
    rain = float(row[rain_tag]) * in2ft * ft2m
    pet = float(row[pet_tag]) * in2ft * ft2m
    spd = [(node, "rainfall", rain) for node in sfr_nodes] + [
        (node, "evaporation", pet) for node in sfr_nodes
    ]
    sfr_spd[n] = spd

LAK datasets

In [ ]:
lak_length_conversion = 1.0  # 3.28081
lak_time_conversion = 86400.0

In [ ]:
lake_ic = 11.3 * ft2m
idx = np.where(lake_cells_vg == 1.0)

lake_map = np.ones(voronoi_grid.ncpl, dtype=int) * -1
lake_map[idx] = 0

(idomain, lakpak_dict, lak_connections) = flopy.mf6.utils.get_lak_connections(
    voronoi_grid,
    lake_map,
    bedleak=0.0013,
)

# add concentration and iflowface to lake data as aux
lake_iflowface = -1.0
lakpak_data = [(0, lake_ic, lakpak_dict[0], 1.0, lake_iflowface)]

In [ ]:
# lake_spd = [
#     (0, "rainfall", rainfall),
#     (0, "evaporation", evaporation),
# ]

lake_spd = {}
rain_tag = "PRCP (Inches)"
pet_tag = "lake et (inches)"
for n in range(nper):
    row = temporal_df.iloc[n]
    rain = float(row[rain_tag]) * in2ft * ft2m
    pet = float(row[pet_tag]) * in2ft * ft2m
    lake_spd[n] = [(0, "rainfall", rain), (0, "evaporation", pet)]

In [ ]:
lake_ic, lakpak_data

In [ ]:
lak_connections[0:10]

DRN datasets

In [ ]:
areas = []
for idx in range(voronoi_grid.ncpl):
    vertices = np.array(voronoi_grid.get_cell_vertices(idx))
    area = Polygon(vertices).area
    areas.append(area)

In [ ]:
drn_iflowface = -1.0
drn_kv = 0.1 * ft2m
drn_depth = 1.0 * ft2m
drn_nodes = []
drn_spd = []
for idx, elev in enumerate(top_vg):
    if lake_cells_vg[idx] > 0:
        drn_nodes.append(idx)
        cond = drn_kv * areas[idx] / drn_depth
        drn_spd.append([(0, idx), elev, cond, -drn_depth, drn_iflowface])
drn_spd[:10]

WEL datasets

In [ ]:
well_points

In [ ]:
well_loc = []
for x, y in well_points:
    well_loc.append(voronoi_grid.intersect(x, y))
well_loc

In [ ]:
# first well is Virginia City well site 2
# second well is Reilly well
# third well is Virginia City well site 1
# boundname = ["VC2", "Reilly", "VC1"]
well_boundnames = ["P3", "P1", "P2"]
rates = [-1900.0, -7600.0, -7600.0]
welspd = [
    [nlay - 1, cellid, rates[idx], well_boundnames[idx]]
    for idx, cellid in enumerate(well_loc)
]
welspd

In [ ]:
p2_loc = tuple(welspd[-1][0:2])
p2_loc

In [ ]:
p2_loc = tuple(welspd[-1][0:2])

## Create the mover data sets

In [ ]:
xc, yc = (
    voronoi_grid.get_xcellcenters_for_layer(0),
    voronoi_grid.get_ycellcenters_for_layer(0),
)
runoff_factor = 0.1

mover_spd = []
for idx, node in enumerate(drn_nodes):
    if runoff_map_vg[node] > 0:
        pak2 = "LAK-1"
        pak2_idx = idx
    else:
        loc = np.array([(xc[node], yc[node])])
        dists = cdist(loc, sfr_points)
        pak2 = "SFR-1"
        pak2_idx = int(np.argmin(dists))
    mover_spd.append(("DRN-1", idx, pak2, pak2_idx, "factor", runoff_factor))

# for idx, v in enumerate(mover_spd):
#     print(idx, v)

## Observations

In [ ]:
obs_df = pd.read_parquet("../data/obs_well_info.parquet.gzip")
obs_df

In [ ]:
wt_locs = []
aq_locs = []
for idx, (x, y) in enumerate(obs_df["xy"]):
    node = voronoi_grid.intersect(x * ft2m, y * ft2m)
    wt_locs.append((0, node))
    aq_locs.append((int(obs_df["aq_layer"].iloc[idx]), node))

In [ ]:
obs_loc = {}
obs_loc[f"{name}.gwf.wt.csv"] = [
    (f"wt{idx + 1:02d}", "HEAD", (k, node)) for idx, (k, node) in enumerate(wt_locs)
]

In [ ]:
obs_loc[f"{name}.gwf.aq.csv"] = [
    (f"aq{idx + 1:02d}", "HEAD", (k, node)) for idx, (k, node) in enumerate(aq_locs)
]

## Plot a series of grids

In [ ]:
def add_subdomain(ax, xll, xur, yll, yur, lw=0.75, color="black", text="B"):
    x = [xll, xll, xur, xur, xll]
    y = [yll, yur, yur, yll, yll]
    ax.plot(x, y, lw=lw, color=color)
    styles.add_text(
        ax=ax,
        text=text,
        x=xll,
        y=yur,
        color=color,
        transform=False,
        ha="right",
        va="bottom",
        italic=False,
    )

In [ ]:
def add_nodes(ax, xll, xur, yll, yur, edge=20.0, text_offset=2.0):
    for idx, (x, y) in enumerate(zip(xcv, ycv)):
        if x > xll and x < xur and y > yll and y < yur:
            ax.plot(x, y, marker=".", markersize=4, color="black")
            if x > xll + edge and x < xur - edge and y > yll and y < yur - edge:
                xt = x + 4 * text_offset
                yt = y + text_offset
                styles.add_text(
                    ax=ax,
                    x=xt,
                    y=yt,
                    text=f"{idx + 1}",
                    bold=False,
                    transform=False,
                    ha="center",
                    va="bottom",
                    fontsize=5,
                )

In [ ]:
def set_ticklabels(
    ax,
    fmt="{:.1f}",
    skip_xticklabels=False,
    skip_yticklabels=False,
    skip_xlabel=False,
    skip_ylabel=False,
    xticks=None,
    yticks=None,
):
    if xticks is None:
        labels = [ax.get_xticks().tolist()]
    else:
        ax.set_xticks(xticks, labels=[str(value) for value in xticks])
        labels = [xticks]

    if yticks is None:
        labels += [ax.get_yticks().tolist()]
    else:
        ax.set_yticks(yticks, labels=[str(value) for value in yticks])
        labels += [yticks]

    for idx, label in enumerate(labels):
        for jdx, value in enumerate(label):
            labels[idx][jdx] = fmt.format(float(value) / 1000.0)

    if skip_xticklabels:
        ax.set_xticklabels([])
    else:
        ax.xaxis.set_major_locator(mticker.FixedLocator(ax.get_xticks()))
        ax.set_xticklabels(labels[0])

    if skip_yticklabels:
        ax.set_yticklabels([])
    else:
        ax.yaxis.set_major_locator(mticker.FixedLocator(ax.get_yticks()))
        ax.set_yticklabels(labels[1])

    if not skip_xlabel:
        ax.set_xlabel("x position (km)")
    if not skip_ylabel:
        ax.set_ylabel("y position (km)")

In [ ]:
def plot_well_labels(ax):
    for xy, name in zip(well_points, well_boundnames):
        styles.add_annotation(
            ax=ax,
            text=name,
            xy=xy,
            xytext=(-15, 10),
            bold=False,
            textcoords="offset points",
            arrowprops=arrowprops,
        )
    return

In [ ]:
def plot_feature_labels(ax):
    styles.add_text(
        ax=ax,
        text="Blue\nLake",
        x=610,
        y=5000.0,
        transform=False,
        bold=False,
        ha="center",
        va="center",
    )
    styles.add_text(
        ax=ax,
        text="Straight River",
        x=1425,
        y=1500.0,
        transform=False,
        bold=False,
        va="center",
        ha="center",
        rotation=90,
    )
    plot_well_labels(ax)
    return

A few coordinates for plotting

In [ ]:
xcv, ycv = voronoi_grid.xcellcenters, voronoi_grid.ycellcenters

In [ ]:
x0 = x1 = sg[:, 0].min()
y0, y1 = sg[:, 1].max(), sg[:, 1].min()
x0, y0, x1, y1

## Build the Model

### flow model

In [ ]:
ws = f"{base_ws}"
sim = flopy.mf6.MFSimulation(sim_ws=ws, sim_name=name, write_headers=False)
tdis = flopy.mf6.ModflowTdis(
    sim,
    time_units="days",
    nper=nper,
    perioddata=tdis_ds,
)
ims = flopy.mf6.ModflowIms(
    sim,
    print_option="all",
    complexity="complex",
    linear_acceleration="bicgstab",
    outer_maximum=500,
    outer_dvclose=1e-3,
    inner_maximum=100,
    inner_dvclose=1e-6,
)
gwf = flopy.mf6.ModflowGwf(
    sim,
    modelname=name,
    save_flows=True,
    newtonoptions="NEWTON UNDER_RELAXATION",
)
dis = flopy.mf6.ModflowGwfdisv(
    gwf,
    length_units="meters",
    nlay=nlay,
    ncpl=vor.ncpl,
    nvert=vor.nverts,
    top=top_vg,
    botm=botm,
    vertices=vor.get_disv_gridprops()["vertices"],
    cell2d=vor.get_disv_gridprops()["cell2d"],
    idomain=idomain_vg,
)
ic = flopy.mf6.ModflowGwfic(gwf, strt=11.0)
npf = flopy.mf6.ModflowGwfnpf(
    gwf,
    xt3doptions=True,
    save_specific_discharge=True,
    save_saturation=True,
    icelltype=icelltype_vg,
    k=k11_vg,
    k33=k33_vg,
)
sto = flopy.mf6.ModflowGwfsto(
    gwf,
    iconvert=icelltype_vg,
    ss=ss_vg,
    sy=porosity_vg,
    steady_state={0: True},
    transient={1: True},
)
rch = flopy.mf6.ModflowGwfrcha(
    gwf,
    pname="rch-1",
    recharge=recharge_spd,
    auxiliary=["iflowface"],
    aux={0: [-1]},
)
evt = flopy.mf6.ModflowGwfevta(
    gwf,
    surface=top_vg,
    rate=evt_spd,
    depth=1.0,
    auxiliary=["iflowface"],
    aux={0: [-1]},
)
wel = flopy.mf6.ModflowGwfwel(
    gwf,
    pname="WEL-1",
    maxbound=len(welspd),
    stress_period_data={1: welspd},
    boundnames=True,
)
drn = flopy.mf6.ModflowGwfdrn(
    gwf,
    mover=True,
    auxiliary=["depth", "iflowface"],
    auxdepthname="depth",
    maxbound=len(drn_spd),
    stress_period_data={0: drn_spd},
    pname="DRN-1",
)
sfr = flopy.mf6.ModflowGwfsfr(
    gwf,
    pname="SFR-1",
    mover=True,
    print_stage=True,
    print_flows=True,
    auxiliary=["iflowface"],
    time_conversion=sfr_time_conversion,
    stage_filerecord=f"{name}.sfr.stage.bin",
    budget_filerecord=f"{name}.sfr.cbc",
    nreaches=len(sfrpak_data),
    packagedata=sfrpak_data,
    connectiondata=sfr_conn,
    perioddata=sfr_spd,
)
lak = flopy.mf6.ModflowGwflak(
    gwf,
    mover=True,
    pname="LAK-1",
    time_conversion=lak_time_conversion,
    length_conversion=lak_length_conversion,
    auxiliary=["concentration", "iflowface"],
    print_stage=True,
    print_flows=True,
    stage_filerecord=f"{name}.lak.stage.bin",
    budget_filerecord=f"{name}.lak.cbc",
    nlakes=1,
    packagedata=lakpak_data,
    connectiondata=lak_connections,
    perioddata=lake_spd,
)
mvr = flopy.mf6.ModflowGwfmvr(
    gwf,
    pname="MVR-1",
    maxpackages=3,
    packages=[["DRN-1"], ["SFR-1"], ["LAK-1"]],
    maxmvr=len(mover_spd),
    perioddata={0: mover_spd},
)
oc = flopy.mf6.ModflowGwfoc(
    gwf,
    head_filerecord=name + ".hds",
    budget_filerecord=name + ".cbc",
    saverecord=[("HEAD", "ALL"), ("BUDGET", "ALL")],
    printrecord=[("BUDGET", "ALL")],
)
gwf_obs = flopy.mf6.ModflowUtlobs(
    gwf,
    pname="GWF-OBS",
    print_input=True,
    continuous=obs_loc,
    filename=f"{name}.gwf.obs",
)
# write the model datasets
sim.write_simulation()

# run the model
success = sim.run_simulation()

### process model results

In [ ]:
output_times = gwf.output.head().get_times()
totim = output_times[0]

In [ ]:
lake_stage = float(gwf.lak.output.stage().get_data(totim=totim).squeeze())
lake_stage

In [ ]:
lake_stage_vg = np.full(vor.ncpl, 1e30, dtype=float)
idx = (lake_stage > top_vg) & (lake_cells_vg > 0)
lake_stage_vg[idx] = lake_stage

In [ ]:
head = gwf.output.head().get_data(totim=totim).squeeze()
head.shape, head.min(), head.max()

In [ ]:
cbc = gwf.output.budget()
cbc.list_unique_records(), cbc.list_unique_packages()

In [ ]:
spdis = cbc.get_data(text="DATA-SPDIS", totim=totim)[0]
qx, qy, qz = flopy.utils.postprocessing.get_specific_discharge(spdis, gwf, head=head)
qx.shape

In [ ]:
lake_q = cbc.get_data(text="LAK", full3D=True)[0]
lake_q.shape

In [ ]:
lake_q_dir = np.zeros(lake_q.shape, dtype=int)
lake_q_dir[lake_q < 0.0] = -1
lake_q_dir[lake_q > 0.0] = 1

#### plot top of model and results

In [ ]:
top_range = (0, 20)
top_levels = np.arange(0, 25, 5)
head_range = (-1, 5)
head_levels = np.arange(-5, head_range[1] + 1, 1)

extent = gwf.modelgrid.extent
xA = well_points[-1][0]
yB = 10500 * ft2m
xsA_pts = ((xA, extent[3]), (xA, extent[2]))
xsB_pts = ((extent[0], yB), (extent[1], yB))
xsA_line = ((xsA_pts[0][0], xsA_pts[1][0]), (xsA_pts[0][1], xsA_pts[1][1]))
xsB_line = ((xsB_pts[0][0], xsB_pts[1][0]), (xsB_pts[0][1], xsB_pts[1][1]))

In [ ]:
xsA_line, xsA_pts

Plot river mapping

In [ ]:
dv = 100.0  # m

with styles.USGSMap():
    fig = plt.figure(figsize=two_panel_figsize, constrained_layout=False)
    gs = gridspec.GridSpec(ncols=10, nrows=24, figure=fig)
    ax0 = fig.add_subplot(gs[:, :5])
    ax2 = fig.add_subplot(gs[9:17, 5:])
    ax1 = fig.add_subplot(gs[0:8, 5:])  # , sharex=ax2)
    # ax1.get_shared_x_axes().join(ax1, ax2)
    ax3 = fig.add_subplot(gs[17:, 5:])

    ax = ax0
    ax.set_aspect("equal", "box")
    styles.heading(ax=ax, idx=0)
    mm = flopy.plot.PlotMapView(modelgrid=voronoi_grid, ax=ax)
    mm.plot_array(
        clay_location_vg,
        masked_values=[
            0,
        ],
        cmap=clay_cmap,
        alpha=0.5,
    )
    mm.plot_array(
        lake_cells_vg,
        masked_values=[
            0,
        ],
        cmap=lake_cmap,
        alpha=0.5,
    )
    mm.plot_grid(**grid_dict)
    plot_river(ax)
    plot_wells(ax, ms=3)
    plot_feature_labels(ax)

    xticks = np.arange(mm.extent[0], mm.extent[1], 1000.0).tolist()
    yticks = np.arange(mm.extent[2], mm.extent[3], 1000.0).tolist()
    set_ticklabels(ax, fmt="{:.0f}", xticks=xticks, yticks=yticks)

    ax = ax1
    ax.set_aspect("equal", "box")
    plt.setp(ax.get_xticklabels(), visible=False)
    styles.heading(ax=ax, idx=1)

    xll, xur = x0 - dv, x1 + dv
    yll, yur = y0 - 2.0 * dv, y0
    xticks = np.arange(xll, xur + 100, 100.0).tolist()
    yticks = np.arange(yll, yur + 100, 100.0).tolist()
    add_subdomain(ax0, xll, xur, yll, yur, text="B")

    mm = flopy.plot.PlotMapView(
        modelgrid=voronoi_grid, ax=ax, extent=(xll, xur, yll, yur)
    )
    mm.plot_array(
        sfr_plt_array,
        lw=0,
        alpha=0.5,
        masked_values=[0],
        cmap=intersection_cmap,
    )
    mm.plot_grid(**grid_dict)
    plot_river(ax)
    add_nodes(ax, xll, xur, yll, yur)
    set_ticklabels(ax, yticks=yticks, skip_xlabel=True)

    ax = ax2
    ax.set_aspect("equal", "box")
    styles.heading(ax=ax, idx=2)

    xll, xur = x0 - dv, x1 + dv
    yll, yur = y1, y1 + 2.0 * dv
    yticks = np.arange(yll, yur + 100.0, 100.0).tolist()
    add_subdomain(ax0, xll, xur, yll, yur, text="C")

    mm = flopy.plot.PlotMapView(
        modelgrid=voronoi_grid, ax=ax, extent=(xll, xur, yll, yur)
    )
    mm.plot_array(
        sfr_plt_array,
        lw=0,
        alpha=0.5,
        masked_values=[0],
        cmap=intersection_cmap,
    )
    mm.plot_grid(**grid_dict)
    plot_river(ax)
    add_nodes(ax, xll, xur, yll, yur)
    set_ticklabels(ax, xticks=xticks, yticks=yticks)
    # print(ax.get_xlim(), ax.get_ylim())

    # legend
    ax = ax3
    xy0 = (-100, -100)
    ax.set_ylim(0, 1)
    ax.set_axis_off()

    # fake data to set up legend
    ax.plot(
        xy0,
        xy0,
        lw=0.0,
        marker=".",
        ms=5,
        mfc="black",
        mec="none",
        mew=0.0,
        label="Cell centroid",
    )
    ax.plot(
        xy0,
        xy0,
        lw=0.0,
        marker=".",
        ms=5,
        mfc="red",
        mec="none",
        mew=0.0,
        label="Well",
    )
    ax.plot(
        xy0,
        xy0,
        lw=0.0,
        marker="s",
        mfc="cyan",
        mec="black",
        mew=0.5,
        alpha=0.5,
        label="Lake",
    )
    ax.plot(
        xy0,
        xy0,
        lw=0.0,
        marker="s",
        mfc="brown",
        mec="black",
        mew=0.5,
        alpha=0.5,
        label="Confining unit",
    )
    ax.axhline(xy0[0], **river_dict, label="River")
    ax.plot(
        xy0,
        xy0,
        lw=0.0,
        marker="s",
        mfc=intersection_rgba,
        mec="black",
        mew=0.5,
        alpha=0.5,
        label="SFR cells",
    )
    # ax.axhline(xy0[0], lw=1, ls=":", color="red", label="Cross-section\nline")
    styles.graph_legend(
        ax,
        ncol=2,
        loc="lower center",
        labelspacing=0.1,
        columnspacing=0.6,
        handletextpad=0.3,
    )

    # save the figure
    if save_figures:
        fpth = fig_path / "sv_voronoi_river_discretization.pdf"
        plt.savefig(fpth, dpi=300)

    plt.show()

In [ ]:
xlims = (voronoi_grid.extent[0], voronoi_grid.extent[1])
xticks = np.arange(xlims[0], xlims[1], 1000.0).tolist()
ylims = (voronoi_grid.extent[2], voronoi_grid.extent[3])
yticks = np.arange(ylims[0], ylims[1], 1000.0).tolist()
xlims, xticks, ylims, yticks

p2_loc = tuple(welspd[-1][0:2])
p2_z = gwf.modelgrid.zcellcenters[p2_loc]

In [ ]:
with styles.USGSMap():
    fig = plt.figure(figsize=two_panel_figsize, constrained_layout=True)
    gs = gridspec.GridSpec(ncols=2, nrows=24, figure=fig)
    ax0 = fig.add_subplot(gs[:22, 0])
    ax1 = fig.add_subplot(gs[:22, 1])
    ax2 = fig.add_subplot(gs[22:, :])

    xticks = np.arange(mm.extent[0], mm.extent[1], 1000.0).tolist()
    yticks = np.arange(mm.extent[2], mm.extent[3], 1000.0).tolist()

    for ax in (ax0, ax1):
        ax.set_xlim(xlims)
        ax.set_xticks(xticks)
        ax.set_ylim(ylims)
        ax.set_yticks(yticks)
        ax.set_aspect("equal", "box")

    # topography
    ax = ax0
    styles.heading(ax=ax, idx=0)
    mm = flopy.plot.PlotMapView(
        model=gwf,
        ax=ax,
        extent=voronoi_grid.extent,
    )
    cb = mm.plot_array(top_vg, vmin=top_range[0], vmax=top_range[1])
    mm.plot_grid(**grid_dict)
    plot_wells(ax=ax, ms=3)
    plot_river(ax=ax)
    plot_lake(ax=ax)
    cs = mm.contour_array(
        top_vg,
        **sv_contour_dict,
        levels=top_levels,
    )
    ax.clabel(
        cs,
        **clabel_dict,
    )
    set_ticklabels(ax, fmt="{:.0f}")

    # topography colorbar
    cbar = plt.colorbar(cb, ax=ax, orientation="horizontal", shrink=0.65)
    cbar.ax.tick_params(
        labelsize=5,
        labelcolor="black",
        color="black",
        length=6,
        pad=2,
    )
    cbar.ax.set_title(
        "Elevation (m)",
        pad=2.5,
        loc="left",
        fontdict=font_dict,
    )

    # cross-section lines
    dlabel = 400
    ax.plot(
        # [7750.0 * ft2m, 7750.0 * ft2m],
        xsA_line[0],
        xsA_line[1],
        [voronoi_grid.extent[3], 0],
        lw=1,
        ls=":",
        color="red",
    )
    styles.add_annotation(
        ax=ax,
        text="A",
        # xy=(7750 * ft2m, voronoi_grid.extent[3]),
        xy=xsA_pts[0],
        xytext=(-15, -15),
        textcoords="offset points",
        arrowprops=arrowprops,
        bold=True,
        color="red",
    )
    styles.add_annotation(
        ax=ax,
        text="A'",
        # xy=(7750 * ft2m, 0),
        xy=xsA_pts[1],
        xytext=(15, 10),
        textcoords="offset points",
        arrowprops=arrowprops,
        bold=True,
        color="red",
    )
    mm.ax.plot(
        # [0, voronoi_grid.extent[1]], [10500 * ft2m, 10500 * ft2m],
        xsB_line[0],
        xsB_line[1],
        lw=1,
        ls=":",
        color="red",
    )
    styles.add_annotation(
        ax=ax,
        text="B",
        # xy=(0, 10500 * ft2m),
        xy=xsB_pts[0],
        xytext=(15, 15),
        textcoords="offset points",
        arrowprops=arrowprops,
        bold=True,
        color="red",
    )
    styles.add_annotation(
        ax=ax,
        text="B'",
        # xy=(voronoi_grid.extent[1], 10500 * ft2m),
        xy=xsB_pts[1],
        xytext=(-15, 10),
        textcoords="offset points",
        arrowprops=arrowprops,
        bold=True,
        color="red",
    )

    # head
    ax = ax1
    styles.heading(ax=ax, idx=1)
    mm = flopy.plot.PlotMapView(
        model=gwf,
        ax=ax,
        extent=voronoi_grid.extent,
    )
    cb = mm.plot_array(head, vmin=head_range[0], vmax=head_range[1])

    mm.plot_grid(**grid_dict)
    q = mm.plot_vector(qx, qy, normalize=False)
    qk = plt.quiverkey(
        q,
        0.08,
        -0.31,
        1.0,
        label="Specific discharge vector",
        labelsep=0.05,
        labelpos="E",
        labelcolor="black",
        fontproperties={"size": 9, "weight": "bold"},
    )

    plot_wells(ax=ax, ms=3)
    plot_river(ax=ax)
    plot_lake(ax=ax)
    cs = mm.contour_array(
        head,
        **sv_contour_dict,
        levels=head_levels,
    )
    ax.clabel(
        cs,
        **clabel_dict,
    )
    set_ticklabels(ax, fmt="{:.0f}", xticks=xticks, yticks=yticks)

    # head colorbar
    cbar = plt.colorbar(cb, ax=ax, orientation="horizontal", shrink=0.65)
    cbar.ax.tick_params(
        labelsize=5,
        labelcolor="black",
        color="black",
        length=6,
        pad=2,
    )
    cbar.ax.set_title(
        "Head (m)",
        pad=2.5,
        loc="left",
        fontdict=font_dict,
    )

    # legend
    ax = ax2
    xy0 = (-100, -100)
    ax.set_xlim(0, 1)
    ax.set_ylim(0, 1)
    ax.set_axis_off()

    # fake data to set up legend
    well_patch = ax.plot(
        xy0,
        xy0,
        lw=0.0,
        marker=".",
        ms=5,
        mfc="red",
        mec="none",
        mew=0.0,
        label="Well",
    )
    lake_patch = ax.axhline(xy0[0], color="cyan", lw=0.5, label="Lake")
    river_patch = ax.axhline(xy0[0], **river_dict, label="River")
    cross_patch = ax.axhline(
        xy0[0], lw=1, ls=":", color="red", label="Cross-section line"
    )
    contour_patch = ax.axhline(xy0[0], **contour_label_dict, label="Contour line (m)")
    styles.graph_legend(
        ax,
        ncol=3,
        loc="lower center",
        labelspacing=0.1,
        columnspacing=0.6,
        handletextpad=0.3,
    )

    if save_figures:
        fpth = fig_path / "sv_voronoi_map.pdf"
        plt.savefig(fpth, dpi=300)

    plt.show()

In [ ]:
ylim = (-80, 30)
with styles.USGSMap():
    fig = plt.figure(figsize=one_panel_figsize, constrained_layout=False)
    gs = gridspec.GridSpec(ncols=100, nrows=21, figure=fig)
    ax0 = fig.add_subplot(gs[:9, :])
    ax1 = fig.add_subplot(gs[11:21, :63])
    ax2 = fig.add_subplot(gs[11:21, 63:88])

    # cross-section A-A'
    ax = ax0
    styles.heading(ax, idx=0)
    fx = flopy.plot.PlotCrossSection(
        model=gwf,
        ax=ax,
        line={"line": xsA_pts},
    )
    fx.plot_array(head, head=head, vmin=head_range[0], vmax=head_range[1])
    fx.plot_grid(**grid_dict)

    fxl = flopy.plot.PlotCrossSection(
        modelgrid=lake_vg_grid,
        ax=ax,
        line={"line": xsA_pts},
    )
    fxl.plot_array(
        lake_stage_vg,
        head=lake_stage_vg,
        cmap=lake_cmap,
        masked_values=[1e30],
        edgecolor="none",
    )

    ax.set_ylim(ylim)
    xs_xticks = np.arange(0, ax.get_xlim()[1], 1000)
    ax.set_xticks(xs_xticks, labels=[f"{x / 1000:.0f}" for x in xs_xticks])
    ax.set_xlabel("Cross-section distance (km)")
    ax.set_ylabel("Elevation (m)")
    xlim = ax.get_xlim()

    # annotation
    styles.add_annotation(
        ax=ax,
        text="A",
        xy=(0, 6),
        xytext=(8, 4),
        textcoords="offset points",
        arrowprops=arrowprops,
        bold=True,
        color="red",
    )
    styles.add_annotation(
        ax=ax,
        text="A'",
        xy=(ax.get_xlim()[1], 3.0),
        xytext=(-15, 12),
        textcoords="offset points",
        arrowprops=arrowprops,
        bold=True,
        color="red",
    )
    styles.add_annotation(
        ax=ax,
        text=well_boundnames[-1],
        xy=(ax.get_xlim()[1] - well_points[-1][1], p2_z),
        xytext=(-25, 15),
        textcoords="offset points",
        arrowprops=arrowprops,
        bold=True,
        color="red",
    )
    styles.add_annotation(
        ax=ax,
        text="Blue Lake",
        xy=(1500, lake_stage),
        xytext=(-25, 10),
        textcoords="offset points",
        arrowprops=arrowprops,
        bold=True,
        color="black",
    )

    # cross-section B-B'
    ax = ax1
    styles.heading(ax, idx=1)
    fx = flopy.plot.PlotCrossSection(
        model=gwf,
        ax=ax,
        line={"line": xsB_pts},
    )
    hv = fx.plot_array(head, head=head, vmin=head_range[0], vmax=head_range[1])
    fx.plot_grid(**grid_dict)

    ax.set_ylim(ylim)
    xs_xticks = np.arange(0, ax.get_xlim()[1], 1000)
    ax.set_xticks(xs_xticks, labels=[f"{x / 1000:.0f}" for x in xs_xticks])
    ax.set_xlabel("Cross-section distance (km)")
    ax.set_ylabel("Elevation (m)")

    # annotation
    styles.add_annotation(
        ax=ax,
        text="B",
        xy=(0, 43 * ft2m),
        xytext=(12, 8),
        textcoords="offset points",
        arrowprops=arrowprops,
        bold=True,
        color="red",
    )
    styles.add_annotation(
        ax=ax,
        text="B'",
        xy=(ax.get_xlim()[1], 12),
        xytext=(-16, 10),
        textcoords="offset points",
        arrowprops=arrowprops,
        bold=True,
        color="red",
    )

    # head colorbar
    ax = ax2
    ax.set_xlim(0, 1)
    ax.set_ylim(0, 1)
    ax.set_axis_off()
    cbar = plt.colorbar(
        hv, ax=ax, orientation="vertical", shrink=0.65, anchor=(0.0, 0.5)
    )
    cbar.ax.tick_params(
        labelsize=5,
        labelcolor="black",
        color="black",
        length=8,
        pad=2,
    )
    cbar.ax.set_title(
        "Head (m)",
        pad=5,
        loc="center",
        fontdict=font_dict,
    )

    fig.align_labels()

    if save_figures:
        fpth = fig_path / "sv_voronoi_xsection.pdf"
        plt.savefig(fpth, dpi=300)

    plt.show()

# Calculate the residuals

In [ ]:
wt_obs = []
for i, j in obs_df["row_column"]:
    tag = "head_layer1"
    wt_obs.append(float(nc_ds[tag].values[(i, j)] * ft2m))

aq_obs = []
for k, (i, j) in zip(obs_df["aq_layer"], obs_df["row_column"]):
    tag = f"head_layer{k + 1}"
    aq_obs.append(float(nc_ds[tag].values[(i, j)] * ft2m))

In [ ]:
sim_wt = np.array([head[idx] for idx in wt_locs])
sim_aq = np.array([head[idx] for idx in aq_locs])

In [ ]:
resid_wt = sim_wt - np.array(wt_obs)
resid_wt

In [ ]:
resid_aq = sim_aq - np.array(aq_obs)
resid_aq

In [ ]:
resid_gb = np.concatenate((resid_wt, resid_aq))

In [ ]:
print(
    f"Water Table Statistics\nMean Error: {resid_wt.mean()} ft.\nRMSE:       {np.sqrt((resid_wt**2).sum()) / resid_wt.shape[0]} ft."
)

In [ ]:
print(
    f"Lower Aquifer Statistics\nMean Error: {resid_aq.mean()} ft.\nRMSE:       {np.sqrt((resid_aq**2).sum()) / resid_aq.shape[0]} ft."
)

In [ ]:
print(
    f"Global Statistics\nMean Error: {resid_gb.mean()} ft.\nRMSE:       {np.sqrt((resid_gb**2).sum()) / resid_gb.shape[0]} ft."
)